# Central Difference Method

In [9]:
import numpy as np
import pandas as pd

## Initial Parameters

In [10]:
k = 90 *1000000           #N/m
m = 489.60 * 1000         #kg
dr = 0.05 
Wn = np.sqrt(k / m)       #rad/s 
Tn = 2 * np.pi / Wn       #s 
c = 2 * m * dr * Wn       #N.sec/m

print(f'Stifness, k = {k:.2e} N/m\n'
      f'mass, m = {m:.2e} kg\n'
      f'Damping Ratio, r = {dr:.3f}\n'
      f'Natural Frequency, Wn = {Wn:.2f} rad/s\n'
      f'Natural Period, Tn = {Tn:.3f} s\n'
      f'Damping Coefficient, c = {c:.2f} N.s/m')

Stifness, k = 9.00e+07 N/m
mass, m = 4.90e+05 kg
Damping Ratio, r = 0.050
Natural Frequency, Wn = 13.56 rad/s
Natural Period, Tn = 0.463 s
Damping Coefficient, c = 663807.20 N.s/m


## Loading El Centro Dataset

In [11]:
elcentro_df = pd.read_csv('Data_Elcentro.csv')       #Replace with your file path

## Initial Calculations

In [12]:
#At i = 0
g = 9.81                                   #m/s^2

#Extracting Ground Acceleration Factor 
ground_acc_factors_g = elcentro_df['acceleration'].values / 100        #converts values from cm/s^2 to m/s^2
print(f'Initial Ground Acceleration Factor,g = {ground_acc_factors_g[0]} m/s^2')            

u_0 = 0           #Initial Displacement, m
v_0 = 0           #Initial Velocity, m/s

#Calculating Initial Force
p_values = -ground_acc_factors_g * m * g
p_0 = p_values[0]                               #N
print(f'Initial Load, p_0 = {p_0} N') 

#Calculating the Initial Acceleration
a_0 = (p_0 - (c*v_0) - (k * u_0)) /m            
print(f'Initial Displacement, u_0 = {u_0:.4f} m\n'
      f'Initial Velocity, v_0 = {v_0:.4f} m/s\n'
      f'Initial Acceleration = {a_0} m/s^2')

#Calculating u-1
delta_t = ground_acc_factors_g[1] - ground_acc_factors_g[0]  #s
u_minus_1 = u_0 - (v_0 * delta_t) + ((a_0 * delta_t**2)/2)
print(f'u-1 = {u_minus_1:.11f} m')

#Calculating k hat
k_hat = (m / delta_t**2) + (c / (2 * delta_t ))
print(f'k_hat = {k_hat:.4g} kg/s^2')

#Calculating a
a = (m / delta_t**2) - (c / (2 * delta_t))
print(f'a = {a:.4g}')

#Calculating b
b = k - (2 * m / delta_t**2)
print(f'b = {b:.4g}')
  
p_hat_0 = p_0 - (a * u_minus_1) - (b * u_0)
print(f'p_hat_0 = {p_hat_0:.4f} N')

#Calculating u_1
u_1 = p_hat_0 / k_hat
print(f'u_1 = {u_1:.11f} m')

Initial Ground Acceleration Factor,g = 6.3e-05 m/s^2
Initial Load, p_0 = -302.587488 N
Initial Displacement, u_0 = 0.0000 m
Initial Velocity, v_0 = 0.0000 m/s
Initial Acceleration = -0.00061803 m/s^2
u-1 = -0.00000000000 m
k_hat = 6.919e+14 kg/s^2
a = 6.92e+14
b = -1.384e+15
p_hat_0 = -151.2910 N
u_1 = -0.00000000000 m


## Initializing Dataframe

In [13]:
values = ['t_i', 'P_i', 'u_i_minus_1', 'u_i', 'P^i', 'u_i_plus_1']
data = pd.DataFrame(columns = values)

## Storing Values from Initial Calculations in DataFrame

In [14]:
data['t_i'] = elcentro_df['time'].values
data['P_i'] = p_values
data['u_i_minus_1'] = u_minus_1
data['u_i'] = a_0
data['P^i'] = p_hat_0
data['u_i_plus_1'] = u_1

In [15]:
data.head()

,t_i,P_i,u_i_minus_1,u_i,P^i,u_i_plus_1
0,0.00,-302.587488,-2.186467e-13,-0.000618,-151.291016,-2.186467e-13
1,0.02,-174.828326,-2.186467e-13,-0.000618,-151.291016,-2.186467e-13
2,0.04,-47.549462,-2.186467e-13,-0.000618,-151.291016,-2.186467e-13
3,0.06,-205.567373,-2.186467e-13,-0.000618,-151.291016,-2.186467e-13
4,0.08,-364.065581,-2.186467e-13,-0.000618,-151.291016,-2.186467e-13


In [ ]:
for i in range(1, len(data)):           #Iteration starts from i = 1
    data.loc[i, 'u_i_minus_1'] = data.loc[i-1, 'u_i']
    data.loc[i, 'u_1'] = data.loc[i-1, 'u_i_plus_1']

    #Calculating P^i
    data.loc[i, 'P^i'] = data.loc[i, 'P_i'] - ((a * data.loc[i, 'u_i_minus_1']) + (b * data.loc[i, 'u_i']))

    #Calculating u_i_plus_1
    data.loc[i, 'u_i_plus_1'] = data.loc[i, 'P^i'] / k_hat